In [1]:
import pandas as pd
import string
import numpy as np
import json
import os

import keras
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.utils import np_utils
# import keras.utils as ku

import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

In [2]:
import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

No GPU found


In [3]:
# print("GPU:", len(tf.config.experimental.list_physical_devices('GPU')))


In [4]:
import tensorflow as tf 
tf.__version__

'2.9.1'

In [5]:
# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# except:
#   # Invalid device or cannot modify virtual devices once initialized.
#   pass
#config = tf.ConfigProto(device_count = {'GPU':0})
#sess = tf.Session(config=config)

In [6]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)

In [7]:
#load all the datasets 
df1 = pd.read_csv("CAvideos.csv")
df2 = pd.read_csv("GBvideos.csv")
df3 = pd.read_csv("INvideos.csv")
df4 = pd.read_csv ("USvideos.csv")


#load the datasets containing the category names
data1 = json.load(open('CAvideos_id.json'))
data2 = json.load(open('GBvideos_id.json'))
data3 = json.load(open('INvideos_id.json'))
data4 = json.load(open('USvideos_id.json'))

C:\Users\shiva\AppData\Local\Temp\ipykernel_12272\4022661005.py:5: DtypeWarning: Columns (3,6,7,8,9,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df4 = pd.read_csv ("USvideos.csv")


In [8]:
def category_extractor(data):
    i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    i_d = list(map(int, i_d))
    category = zip(i_d, title)
    category = dict(category)
    return category

#create a new category column by mapping the category names to their id
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))
df4['category_title'] = df4['category_id'].map(category_extractor(data4))


#join the dataframes
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
print(df.shape)
#drop rows based on duplicate videos
df = df.drop_duplicates('video_id')

#collect only titles of entertainment videos
#feel free to use any category of video that you want
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()
print(df.shape)



#remove punctuations and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

(675356, 15)
(112370, 15)


In [9]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
  #get tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
 
  #convert to sequence of tokens
  input_sequences = []
  for line in corpus:
      token_list = tokenizer.texts_to_sequences([line])[0]
      for i in range(1, len(token_list)):
          n_gram_sequence = token_list[:i+1]
          input_sequences.append(n_gram_sequence)
 
  return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [10]:
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = keras.utils.np_utils.to_categorical(label, num_classes = total_words)
  return predictors, label, max_sequence_len
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [11]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()
 
  # Add Input Embedding Layer
  model.add(Embedding(total_words, 10, input_length=input_len))
 
  # Add Hidden Layer 1 — LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))
 
  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
 
  return model
model = create_model(max_sequence_len, total_words)

checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(predictors, label, epochs=100, verbose=1,callbacks=[cp_callback])

Epoch 1/100
8202/8204 [============================>.] - ETA: 0s - loss: 7.7791
Epoch 1: saving model to cp.ckpt
8204/8204 [==============================] - 190s 23ms/step - loss: 7.7789
Epoch 2/100
8203/8204 [============================>.] - ETA: 0s - loss: 6.6891
Epoch 2: saving model to cp.ckpt
8204/8204 [==============================] - 191s 23ms/step - loss: 6.6891
Epoch 3/100
8203/8204 [============================>.] - ETA: 0s - loss: 6.0692
Epoch 3: saving model to cp.ckpt
8204/8204 [==============================] - 188s 23ms/step - loss: 6.0692
Epoch 4/100
8203/8204 [============================>.] - ETA: 0s - loss: 5.6214
Epoch 4: saving model to cp.ckpt
8204/8204 [==============================] - 187s 23ms/step - loss: 5.6214
Epoch 5/100
8202/8204 [============================>.] - ETA: 0s - loss: 5.2774
Epoch 5: saving model to cp.ckpt
8204/8204 [==============================] - 196s 24ms/step - loss: 5.2774
Epoch 6/100
8202/8204 [============================>.] - ETA

8204/8204 [==============================] - ETA: 0s - loss: 3.0316
Epoch 87: saving model to cp.ckpt
8204/8204 [==============================] - 193s 24ms/step - loss: 3.0316
Epoch 88/100
8202/8204 [============================>.] - ETA: 0s - loss: 3.0295
Epoch 88: saving model to cp.ckpt
8204/8204 [==============================] - 196s 24ms/step - loss: 3.0295
Epoch 89/100
8204/8204 [==============================] - ETA: 0s - loss: 3.0294
Epoch 89: saving model to cp.ckpt
8204/8204 [==============================] - 196s 24ms/step - loss: 3.0294
Epoch 90/100
8204/8204 [==============================] - ETA: 0s - loss: 3.0281
Epoch 90: saving model to cp.ckpt
8204/8204 [==============================] - 196s 24ms/step - loss: 3.0281
Epoch 91/100
8204/8204 [==============================] - ETA: 0s - loss: 3.0239
Epoch 91: saving model to cp.ckpt
8204/8204 [==============================] - 194s 24ms/step - loss: 3.0239
Epoch 92/100
8203/8204 [============================>.] - ETA: 

In [12]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,  padding='pre')
        predict_x = model.predict(token_list, verbose=0)
        predicted = np.argmax(predict_x,axis=1)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [13]:
model.save('backup_model.h5')

In [14]:
tf.__version__

'2.9.1'

In [15]:
print(generate_text("spiderman", 5, model, max_sequence_len))

Spiderman No Way Home Official Trailer


In [21]:
print(generate_text("superman", 5, model, max_sequence_len))

Superman Meets Water Top 7 Best


In [17]:
print(generate_text("", 5, model, max_sequence_len))

 Diaries Episode 1 The Slow


In [18]:
df.shape

(112370, 15)

In [22]:
print(generate_text("Johnny", 5, model, max_sequence_len))

Johnny Depp Talks To Kanye West


In [26]:
print(generate_text("Will", 4, model, max_sequence_len))

Will Smith On The Internet


In [27]:
print(generate_text("slapped", 5, model, max_sequence_len))

Slapped Diaries Episode 1 The Slow


In [28]:
print(max_sequence_len)

27


In [29]:
print(generate_text("Pikachu", 5, model, max_sequence_len))

Pikachu Jam On Trump Jones Gq


In [30]:
print(generate_text("pika", 5, model, max_sequence_len))

Pika Diaries Episode 1 The Slow


In [31]:
print(generate_text("Ryan", 5, model, max_sequence_len))

Ryan Reynolds Doesnt Know Anything About


In [32]:
print(generate_text("Amber", 5, model, max_sequence_len))

Amber Rock For The First Time


In [33]:
print(generate_text("Tom", 5, model, max_sequence_len))

Tom Holland Zendaya Jacob Batalon And


In [34]:
print(generate_text("Zendaya", 5, model, max_sequence_len))

Zendaya Is A Good Face In


In [35]:
print(generate_text("Marvel", 5, model, max_sequence_len))

Marvel Studios Avengers Infinity War Trailer


In [36]:
print(generate_text("Robert", 5, model, max_sequence_len))

Robert A Big Boy The Worst


In [37]:
print(generate_text("Battle", 5, model, max_sequence_len))

Battle Of Van First Impressions Friendship


In [38]:
print(generate_text("Friends", 5, model, max_sequence_len))

Friends With Benefits Episode 2 Telugu
